In [3]:
import numpy as np
import sys
import scipy
from scipy.optimize import curve_fit
from scipy.stats import unitary_group
import time as t
import matplotlib.pyplot as plt
np.set_printoptions(precision=2, linewidth=100)


In [5]:
class DecisionTree():
    def __init__(self, data = None):
        self.nodes = []
        self.data = data
        
#define the decision tree of the game
class LoanGameTree():
    def __init__(self):
        self.root = DecisionTree(["lender","creation -> funding -> giving"])
        self.root.nodes.append(DecisionTree(["renter","lender cheat give"]))
        self.root.nodes[0].nodes.append(DecisionTree(["lender","lender cheat give"]))
        self.root.nodes[0].nodes[0].nodes.append(DecisionTree(["exit","blameCannotGive(0)"]))
        self.root.nodes[0].nodes[0].nodes.append(DecisionTree(["exit","blameRenterRecieved(0)"]))
        self.root.nodes[0].nodes[0].nodes.append(DecisionTree(["exit","abortCouldntGive(0)"]))
        self.root.nodes.append(DecisionTree(["exit","abortTimeoutGiving()"]))
        self.root.nodes.append(DecisionTree(["renter","lender truth give"]))
        self.root.nodes[2].nodes.append(DecisionTree(["lender","renter cheat get"]))
        self.root.nodes[2].nodes[0].nodes.append(DecisionTree(["exit","blameCannotGive(1)"]))
        self.root.nodes[2].nodes[0].nodes.append(DecisionTree(["exit","blameRenterRecieved(1)"]))
        self.root.nodes[2].nodes[0].nodes.append(DecisionTree(["exit","abortCouldntGive(1)"]))
        self.root.nodes[2].nodes.append(DecisionTree(["lender","renter truth get"]))
        self.root.nodes[2].nodes[1].nodes.append(DecisionTree(["exit","abortTimeoutForcePurchase()"]))
        self.root.nodes[2].nodes[1].nodes.append(DecisionTree(["renter","lender try to return"]))
        self.root.nodes[2].nodes[1].nodes[1].nodes.append(DecisionTree(["lender","renter cheat"]))
        self.root.nodes[2].nodes[1].nodes[1].nodes[0].nodes.append(DecisionTree(["exit","blameFalseReturn(1)"]))
        self.root.nodes[2].nodes[1].nodes[1].nodes[0].nodes.append(DecisionTree(["exit","Finalize(1)"]))
        self.root.nodes[2].nodes[1].nodes[1].nodes.append(DecisionTree(["exit","abortTimeoutForcePurchase()"]))
        self.root.nodes[2].nodes[1].nodes[1].nodes.append(DecisionTree(["lender","renter truth"]))
        self.root.nodes[2].nodes[1].nodes[1].nodes[2].nodes.append(DecisionTree(["exit","blameFalseReturn(0)"]))
        self.root.nodes[2].nodes[1].nodes[1].nodes[2].nodes.append(DecisionTree(["exit","Finalize(0)"]))
        
class LenderStrategy():
    def __init__(self):
        self.weightsList = np.zeros((3,8))
        self.runningTotal = 0
        
        
class RenterStrategy():
    def __init__(self):
        self.weightsList = np.zeros((2,3))
        self.runningTotal = 0

In [6]:
players = 10
Beta = 1

lenders = [RenterStrategy() for i in range(players)]
renters = [LenderStrategy() for i in range(players)]

#remember to look up the metropolis equation
